## Background

The FBI collates incident-based data by state, summary data with estimates, and data on specific topics like assaults on law enforcement officers, hate crime, or human trafficking are available for download in CSV files. Data is also available via the Crime Data API access to UCR data. The Uniform Crime Reporting (UCR) Program provided updated data for 2017 on September 24, 2018.

## Data Source

Zip files were individually downloaded from https://crime-data-explorer.fr.cloud.gov/downloads-and-docs and placed into the data/NIBRS folder.  


In [1]:
import os
import pandas as pd
import zipfile
from tqdm import tqdm, tnrange

def normalize_columns(df):
    """Clean up columns to allow for joining across data frames more easily"""
    df.columns = map(str.lower, df.columns)
    if "ff_line_number" in df.columns:
        df = df.drop("ff_line_number", axis=1)
    if "hc_flag" in df.columns:
        df = df.drop("hc_flag", axis=1)
    if "data_year" in df.columns:
        df = df.drop("data_year", axis=1)
    return df

def process_year(year):
    """Processes a single year's worth of CSVs into a single dataframe"""
    zf = zipfile.ZipFile(f'../data/NIBRS/MA-{year}.zip')
    csv_files = [x.filename for x in zf.infolist() if x.filename.endswith(".csv")]
    keys = {file_name: file_name.lower().replace(".csv","").replace("ma/","") for file_name in csv_files}
    data = {keys[file_name]: normalize_columns(pd.read_csv(zf.open(file_name), low_memory=False, encoding='latin')) for file_name in csv_files}
    
    df = data["nibrs_incident"].\
        merge(data["nibrs_offense"], on=["incident_id"], suffixes=(False, False)).\
        merge(data["nibrs_offense_type"], on=["offense_type_id"], suffixes=(False, False)).\
        merge(data["nibrs_arrestee"], on=["incident_id", "offense_type_id"], suffixes=(False, False)).\
        merge(data["nibrs_arrestee_weapon"], on=["arrestee_id"], suffixes=(False, False)).\
        merge(data["nibrs_weapon_type"], on=["weapon_id"], suffixes=(False, False)).\
        merge(data["nibrs_age"], on=["age_id"], suffixes=(False, False)).\
        merge(data["nibrs_arrest_type"], on=["arrest_type_id"], suffixes=(False, False)).\
        merge(data["nibrs_ethnicity"], on=["ethnicity_id"], suffixes=(False, False)).\
        merge(data["nibrs_victim_offense"], on=["offense_id"], suffixes=(False, False)).\
        merge(data["nibrs_victim_injury"], on=["victim_id"], suffixes=(False, False)).\
        merge(data["nibrs_injury"], on=["injury_id"], suffixes=(False, False)).\
        merge(data["agency_participation"], on=["agency_id"], suffixes=(False, False))
    return df

In [4]:
# Loop through and create one large dataframe of all the data from 1995 to 2015
# Still working through formatting changes from 2016 and 2017
df = pd.concat(map(process_year, tnrange(1995, 2016)), sort=True)

In [186]:
df.head()

,age_code,age_id,age_name,age_num,age_range_high_num,age_range_low_num,agency_id,arrest_date,arrest_num,arrest_type_code,...,report_date_flag,resident_code,sex_code,shr_flag,submission_date,under_18_disposition_code,victim_id,weapon_code,weapon_id,weapon_name
0,AG,5,Age in Years,35.0,NaN,NaN,7859,1995-02-16 00:00:00,A9501275,O,...,NaN,R,M,N,NaN,NaN,3572601,01,1,Unarmed
1,AG,5,Age in Years,41.0,NaN,NaN,7859,1995-09-07 00:00:00,A9507452,O,...,NaN,R,M,N,NaN,NaN,3582954,01,1,Unarmed
2,AG,5,Age in Years,37.0,NaN,NaN,7859,1995-10-05 00:00:00,I959092501,O,...,NaN,N,M,N,NaN,NaN,3584222,01,1,Unarmed
3,AG,5,Age in Years,48.0,NaN,NaN,7859,1995-10-02 00:00:00,A9508328,O,...,NaN,R,M,N,NaN,NaN,3584124,01,1,Unarmed
4,AG,5,Age in Years,32.0,NaN,NaN,7859,1995-10-25 00:00:00,A9508959,O,...,NaN,R,F,N,NaN,NaN,3585238,01,1,Unarmed


In [188]:
df.columns

Index(['age_code', 'age_id', 'age_name', 'age_num', 'age_range_high_num',
       'age_range_low_num', 'agency_id', 'arrest_date', 'arrest_num',
       'arrest_type_code', 'arrest_type_id', 'arrest_type_name', 'arrestee_id',
       'arrestee_seq_num', 'attempt_complete_flag', 'cargo_theft_flag',
       'clearance_ind', 'cleared_except_date', 'cleared_except_id',
       'crime_against', 'ct_flag', 'data_home', 'ddocname', 'did',
       'ethnicity_code', 'ethnicity_id', 'ethnicity_name', 'hc_code',
       'incident_date', 'incident_hour', 'incident_id', 'incident_number',
       'incident_status', 'injury_code', 'injury_id', 'injury_name',
       'location_id', 'method_entry_code', 'multiple_indicator',
       'nibrs_arrestee_weapon_id', 'nibrs_month_id', 'num_premises_entered',
       'offense_category_name', 'offense_code', 'offense_group', 'offense_id',
       'offense_name', 'offense_type_id', 'orig_format', 'race_id',
       'report_date_flag', 'resident_code', 'sex_code', 'shr_flag'

In [183]:
df[["arrestee_id", "incident_date"]].\
    groupby(by=['arrestee_id']).\
    agg(["count"]).\
    sort_values(by=("incident_date","count"))

,incident_date
,count
arrestee_id,
690533,1
16615638,1
16615637,1
16615636,1
16615632,1
16615631,1
16615629,1
16615628,1
